# DSCI 100: Project Planning - Tuna Sayin
In this document, I will analyze 2 of the dataset we have (players, sessions) and will explain these datasets. Further on I will select on broad and one specific question and explian why I chose this question. After selecting the questions, for answering the questions bettter, I will visualize the dataset we have to answer according to the questions and then will recommend how to proceed from there on.

Please, run the cell below.

In [ ]:
install.packages(c("tidyverse", "tidymodels", "cowplot", "skimr", "janitor", "lubridate", "ggplot2"))
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)


The downloaded binary packages are in
	/var/folders/m_/1h9y57qn4759pt85y0jmqcv00000gn/T//RtmpBKFz3l/downloaded_packages


##### Loading and Understanding the Datasets
Let's start by loading the datasets, for better practice I will split this part into 2. We will start by discovering first players dataset then sessions to better understand the datasets we have. We will underline the columns of each of the dataset and try to understand the dataset.

In [ ]:
players <- read_csv("data/players.csv")
glimpse(players)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 196
Columns: 7
$ experience   <chr> "Pro", "Veteran", "Veteran", "Amateur", "Regular", "Amate…
$ subscribe    <lgl> TRUE, TRUE, FALSE, TRUE, TRUE, TRUE, TRUE, FALSE, TRUE, T…
$ hashedEmail  <chr> "f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8…
$ played_hours <dbl> 30.3, 3.8, 0.0, 0.7, 0.1, 0.0, 0.0, 0.0, 0.1, 0.0, 1.6, 0…
$ name         <chr> "Morgan", "Christian", "Blake", "Flora", "Kylie", "Adrian…
$ gender       <chr> "Male", "Male", "Male", "Female", "Male", "Female", "Fema…
$ Age          <dbl> 9, 17, 17, 21, 21, 17, 19, 21, 47, 22, 23, 17, 25, 22, 17…


As we can see above, players dataset has 7 columns and 196 rows.

Here is the explanation of the columns in this dataset:
- experinece: Player's experience
- subscribe: Player subribed or not
- hashedEmail: Player's email in hashed version
- played_hours: Hours played by that player
- name: Player's name
- gender: Player's gender
- Age: Player's age

In [ ]:
sessions <- read_csv("data/sessions.csv")
glimpse(sessions)

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 1,535
Columns: 5
$ hashedEmail         <chr> "bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8a…
$ start_time          <chr> "30/06/2024 18:12", "17/06/2024 23:33", "25/07/202…
$ end_time            <chr> "30/06/2024 18:24", "17/06/2024 23:46", "25/07/202…
$ original_start_time <dbl> 1.71977e+12, 1.71867e+12, 1.72193e+12, 1.72188e+12…
$ original_end_time   <dbl> 1.71977e+12, 1.71867e+12, 1.72193e+12, 1.72188e+12…


As we can see above, sessions dataset has 5 columns and 1535 rows.

Here is the explanation of the columns in this dataset:
- hashedEmail: Hashed version of the player's email (key to identify the row)
- start_time: Session start time
- end_time: Session end time
- original_start_time: Start timestamp
- original_end_time: End timestamp

In summary, we have 2 dataset files. The first file ("players.csv"), lists each of the player's personal details that played on the minecraft server. We have their personal details like their age, name, gender, etc.
The second file ("sessions.csv"), has records of every gameplay session for every player that played on the minecraf server. 

By combining these 2 datasets, throughout hashedEmail column we can exmaine how player characteristics relate to engagement behaviours such as playtime or session amount. Discovering this type of relation can help us further predict more variables